In [1]:
!pip install torchtext==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -q datasets

In [4]:

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.data import Field, BucketIterator, Iterator
from torchtext.legacy import data



import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np
import pandas as pd

import random
import math
import time

import datasets
from datasets import load_dataset
from datasets import Dataset

In [5]:
train_dataset = load_dataset('json', data_files = "./conala-train.json")  
test_dataset = load_dataset('json', data_files = "./conala-test.json")  
print("Train dataset - ",train_dataset)
print("Test dataset - ",test_dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Train dataset -  DatasetDict({
    train: Dataset({
        features: ['intent', 'rewritten_intent', 'snippet', 'question_id'],
        num_rows: 2379
    })
})
Test dataset -  DatasetDict({
    train: Dataset({
        features: ['intent', 'rewritten_intent', 'snippet', 'question_id'],
        num_rows: 500
    })
})


In [6]:
def creating_features(raw_data):
    # manually curated data
    input_texts = raw_data['train']['rewritten_intent']
    code_snippets = raw_data['train']["snippet"]
    for i in range(len(input_texts)):
        if(raw_data['train']['rewritten_intent'][i] == None):
            input_texts[i] = raw_data['train']['intent'][i]
    return  input_texts,code_snippets

In [7]:
model_inputs,outputs=creating_features(train_dataset)
df=pd.concat([pd.DataFrame(model_inputs),pd.DataFrame(outputs)],axis=1)
df.columns=['Text','Code']

import numpy as np

np.random.seed(0)
msk = np.random.rand(len(df)) < 0.85 

train_df = df[msk]
val_df = df[~msk]

train_df=train_df.reset_index(drop=True)
val_df=val_df.reset_index(drop=True)

In [8]:


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(train_dataset)
print(val_dataset)

Dataset({
    features: ['Text', 'Code'],
    num_rows: 1989
})
Dataset({
    features: ['Text', 'Code'],
    num_rows: 390
})


In [9]:
from tokenize import tokenize, untokenize
import io


def tokenize_python_code(python_code_str):
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output = []
    for i in range(0, len(python_tokens)):
        tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
    return tokenized_output

In [10]:
tokenize_python_code(df.iloc[497,1])

[(62, 'utf-8'),
 (1, 'sys'),
 (54, '.'),
 (1, 'stdout'),
 (54, '.'),
 (1, 'flush'),
 (54, '('),
 (54, ')'),
 (4, ''),
 (0, '')]

In [11]:
import keyword

print(keyword.kwlist)

['False', 'None', 'True', 'and', 'as', 'assert', 'async', 'await', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']


In [12]:
def augment_tokenize_python_code(python_code_str, mask_factor=0.3):


    var_dict = {} # Dictionary that stores masked variables

    # certain reserved words that should not be treated as normal variables and
    # hence need to be skipped from our variable mask augmentations
    skip_list = ['range', 'enumerate', 'print', 'ord', 'int', 'float', 'zip'
                 'char', 'list', 'dict', 'tuple', 'set', 'len', 'sum', 'min', 'max']
    skip_list.extend(keyword.kwlist)

    var_counter = 1
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output = []

    for i in range(0, len(python_tokens)):
      if python_tokens[i].type == 1 and python_tokens[i].string not in skip_list:
        
        if i>0 and python_tokens[i-1].string in ['def', '.', 'import', 'raise', 'except', 'class']: # avoid masking modules, functions and error literals
          skip_list.append(python_tokens[i].string)
          tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
        elif python_tokens[i].string in var_dict:  # if variable is already masked
          tokenized_output.append((python_tokens[i].type, var_dict[python_tokens[i].string]))
        elif random.uniform(0, 1) > 1-mask_factor: # randomly mask variables
          var_dict[python_tokens[i].string] = 'var_' + str(var_counter)
          var_counter+=1
          tokenized_output.append((python_tokens[i].type, var_dict[python_tokens[i].string]))
        else:
          skip_list.append(python_tokens[i].string)
          tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
      
      else:
        tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
    
    return tokenized_output

In [13]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [14]:

Input = data.Field(tokenize = 'spacy',
            init_token='', 
            eos_token='', 
            lower=True)

Output = data.Field(tokenize = augment_tokenize_python_code,
                    init_token='', 
                    eos_token='', 
                    lower=False)

/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [15]:
fields = [('Input', Input),('Output', Output)]

In [16]:
train_example = []
val_example = []

train_expansion_factor = 10
for j in range(train_expansion_factor):
  for i in range(train_df.shape[0]):
    try:
      ex = data.Example.fromlist([train_df.Text[i], train_df.Code[i]], fields)
      train_example.append(ex)
    except:
      pass

for i in range(val_df.shape[0]):
    try:
        ex = data.Example.fromlist([val_df.Text[i], val_df.Code[i]], fields)
        val_example.append(ex)
    except:
        pass       

In [17]:
train_data = data.Dataset(train_example, fields)
valid_data =  data.Dataset(val_example, fields)

Input.build_vocab(train_data, min_freq = 0)
Output.build_vocab(train_data, min_freq = 0)



In [18]:
Output.vocab

In [19]:
def save_vocab(vocab, path):
    import pickle
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 1000):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [22]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [23]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [24]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention


In [25]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 10000):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention


In [26]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention


In [27]:

class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [28]:
INPUT_DIM = len(Input.vocab)
OUTPUT_DIM = len(Output.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [29]:
len(Output.vocab.__dict__['freqs'])

2534

In [30]:
SRC_PAD_IDX = Input.vocab.stoi[Input.pad_token]
TRG_PAD_IDX = Output.vocab.stoi[Output.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [31]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [32]:
model.apply(initialize_weights);

In [33]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [34]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

class CrossEntropyLoss(nn.CrossEntropyLoss):
    """CrossEntropyLoss - with ability to recieve distrbution as targets, and optional label smoothing"""

    def __init__(self, weight=None, ignore_index=-100, reduction='mean', smooth_eps=None, smooth_dist=None, from_logits=True):
        super(CrossEntropyLoss, self).__init__(weight=weight,
                                               ignore_index=ignore_index, reduction=reduction)
        self.smooth_eps = smooth_eps
        self.smooth_dist = smooth_dist
        self.from_logits = from_logits

    def forward(self, input, target, smooth_dist=None):
        if smooth_dist is None:
            smooth_dist = self.smooth_dist
        return cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index,
                             reduction=self.reduction, smooth_eps=self.smooth_eps,
                             smooth_dist=smooth_dist, from_logits=self.from_logits)


def cross_entropy(inputs, target, weight=None, ignore_index=-100, reduction='mean',
                  smooth_eps=None, smooth_dist=None, from_logits=True):
    """cross entropy loss, with support for target distributions and label smoothing https://arxiv.org/abs/1512.00567"""
    smooth_eps = smooth_eps or 0

    # ordinary log-liklihood - use cross_entropy from nn
    if _is_long(target) and smooth_eps == 0:
        if from_logits:
            return F.cross_entropy(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)
        else:
            return F.nll_loss(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)

    if from_logits:
        # log-softmax of inputs
        lsm = F.log_softmax(inputs, dim=-1)
    else:
        lsm = inputs

    masked_indices = None
    num_classes = inputs.size(-1)

    if _is_long(target) and ignore_index >= 0:
        masked_indices = target.eq(ignore_index)

    if smooth_eps > 0 and smooth_dist is not None:
        if _is_long(target):
            target = onehot(target, num_classes).type_as(inputs)
        if smooth_dist.dim() < target.dim():
            smooth_dist = smooth_dist.unsqueeze(0)
        target.lerp_(smooth_dist, smooth_eps)

    if weight is not None:
        lsm = lsm * weight.unsqueeze(0)

    if _is_long(target):
        eps_sum = smooth_eps / num_classes
        eps_nll = 1. - eps_sum - smooth_eps
        likelihood = lsm.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1)
        loss = -(eps_nll * likelihood + eps_sum * lsm.sum(-1))
    else:
        loss = -(target * lsm).sum(-1)

    if masked_indices is not None:
        loss.masked_fill_(masked_indices, 0)

    if reduction == 'sum':
        loss = loss.sum()
    elif reduction == 'mean':
        if masked_indices is None:
            loss = loss.mean()
        else:
            loss = loss.sum() / float(loss.size(0) - masked_indices.sum())

    return loss


def onehot(indexes, N=None, ignore_index=None):
    """
    Creates a one-representation of indexes with N possible entries
    if N is not specified, it will suit the maximum index appearing.
    indexes is a long-tensor of indexes
    ignore_index will be zero in onehot representation
    """
    if N is None:
        N = indexes.max() + 1
    sz = list(indexes.size())
    output = indexes.new().byte().resize_(*sz, N).zero_()
    output.scatter_(-1, indexes.unsqueeze(-1), 1)
    if ignore_index is not None and ignore_index >= 0:
        output.masked_fill_(indexes.eq(ignore_index).unsqueeze(-1), 0)
    return output

def _is_long(x):
    if hasattr(x, 'data'):
        x = x.data
    return isinstance(x, torch.LongTensor) or isinstance(x, torch.cuda.LongTensor)

In [35]:
def maskNLLLoss(inp, target, mask):
    # print(inp.shape, target.shape, mask.sum())
    nTotal = mask.sum()
    crossEntropy = CrossEntropyLoss(ignore_index = TRG_PAD_IDX, smooth_eps=0.20)
    loss = crossEntropy(inp, target)
    loss = loss.to(device)
    return loss, nTotal.item()

In [36]:
criterion = maskNLLLoss

In [37]:
from tqdm import tqdm

def make_trg_mask(trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != TRG_PAD_IDX).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    print("here")
    n_totals = 0
    print_losses = []
    print("iterator")
    print(iterator)
    for i, batch in tqdm(enumerate(iterator), total=len(iterator)):
        # print(batch)
        loss = 0
        src = batch.Input.permute(1, 0)
        trg = batch.Output.permute(1, 0)
        trg_mask = make_trg_mask(trg)
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        mask_loss, nTotal = criterion(output, trg, trg_mask)
        
        mask_loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        print_losses.append(mask_loss.item() * nTotal)
        n_totals += nTotal
        print(i)


    print("printloss",print_losses)
    print("n_totals",n_totals)
    return sum(print_losses) / n_totals


In [38]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    print("there")
    n_totals = 0
    print_losses = []
    
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator), total=len(iterator)):

            src = batch.Input.permute(1, 0)
            trg = batch.Output.permute(1, 0)
            trg_mask = make_trg_mask(trg)

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            mask_loss, nTotal = criterion(output, trg, trg_mask)

            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

        
    return sum(print_losses) / n_totals

In [39]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_example = []
    val_example = []

    for i in range(train_df.shape[0]):
        try:
            ex = data.Example.fromlist([train_df.Text[i], train_df.Code[i]], fields)
            train_example.append(ex)
        except:
            pass

    for i in range(val_df.shape[0]):
        try:
            ex = data.Example.fromlist([val_df.Text[i], val_df.Code[i]], fields)
            val_example.append(ex)
        except:
            pass       

    train_data = data.Dataset(train_example, fields)
    valid_data =  data.Dataset(val_example, fields)

    BATCH_SIZE = 16
    train_iterator, valid_iterator = BucketIterator.splits((train_data, valid_data), batch_size = BATCH_SIZE, 
                                                                sort_key = lambda x: len(x.Input),
                                                                sort_within_batch=True, device = device)

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

here
iterator


  1%|          | 1/125 [00:00<01:28,  1.41it/s]

0


  2%|▏         | 2/125 [00:01<01:18,  1.56it/s]

1


...

123


100%|██████████| 125/125 [01:20<00:00,  1.55it/s]


124
printloss [23788.62841129303, 28248.923825740814, 30956.919103860855, 16757.7135014534, 18853.819309473038, 13518.328838825226, 8938.06909108162, 15601.60611486435, 13627.214086055756, 14940.046795845032, 26184.194147109985, 41275.64899635315, 23147.819109916687, 19404.631784677505, 29007.90492916107, 26977.79771232605, 26743.908370018005, 19662.07617044449, 30443.494139671326, 39313.13066697121, 26116.44446372986, 33952.773711919785, 16999.158495903015, 24732.40920829773, 19953.424008369446, 14182.375874757767, 32562.84561896324, 39338.97993206978, 27810.568403720856, 53488.05390405655, 29104.91699552536, 27799.37864756584, 22696.347076892853, 39046.43373012543, 14305.665040969849, 22343.241196870804, 17410.919321537018, 10024.856100082397, 24000.828186035156, 23107.525552749634, 25788.67202758789, 19376.444657564163, 14506.094547510147, 27035.163308143616, 25964.62042093277, 14174.829013824463, 27908.78815126419, 21891.25297164917, 28738.923717975616, 30568.060326576233, 13431.37

100%|██████████| 25/25 [00:04<00:00,  5.24it/s]


Epoch: 05 | Time: 1m 25s
	Train Loss: 3.422 | Train PPL:  30.635
	 Val. Loss: 3.664 |  Val. PPL:  39.015


In [41]:
SRC = Input
TRG = Output

In [50]:
model.load_state_dict(torch.load('./model_16heads.pt'))

<All keys matched successfully>

In [44]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 500):
    
    model.eval()
    # print(sentence)
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    # print("here")
    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        # print("done1")
        with torch.no_grad():
            # print("done2")
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
            # print("done3")
        
        # print("here2")
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    # print("here1")
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention


In [46]:
import re
from nltk.translate.bleu_score import sentence_bleu
import warnings
warnings.filterwarnings("ignore")

""" The tokenizer that we use for code submissions, from Wang Ling et al., Latent Predictor Networks for Code Generation (2016)
    @param code: string containing a code snippet
    @return: list of code tokens
"""
def tokenize_for_bleu_eval(code):
    code = re.sub(r'([^A-Za-z0-9_])', r' \1 ', code)
    code = re.sub(r'([a-z])([A-Z])', r'\1 \2', code)
    code = re.sub(r'\s+', ' ', code)
    code = code.replace('"', '`')
    code = code.replace('\'', '`')
    tokens = [t for t in code.split(' ') if t]
    return tokens

""" This scores hypotheses against references using BLEU.
    @param reference_list:  list of ground truth samples
    @param hypothesis_list: list of predictions that a model generates.
    @return: average bleu_score of all the data samples
"""
def evaluate_bleu(reference_list, hypothesis_list):
  bleu_score = 0
  number_of_samples = len(reference_list)
  for index in range(number_of_samples):
    reference_tokens = tokenize_for_bleu_eval(reference_list[index])
    hypothesis_tokens = tokenize_for_bleu_eval(hypothesis_list[index])
    bleu_score += compute_bleu([reference_tokens], hypothesis_tokens)
  return (bleu_score/number_of_samples)*100

def compute_bleu(reference, candidate):
  return sentence_bleu(references=reference, hypothesis=candidate, weights=(1.0, 0.0, 0.0, 0.0))

In [ ]:
test_inputs,reference_output=creating_features(test_dataset)
# inputs=[test_inputs[497]]
# print(inputs)
hypothesis_list=[]
for i,input in enumerate(test_inputs):
  
  input=input.split(" ")
  print(i,input)
  translation, attention = translate_sentence(input, SRC, TRG, model, device)
  # print(translation)
  hypothesis_list.append(untokenize(translation[:-1]).decode('utf-8'))
  # print(hypothesis_list)

bleu = evaluate_bleu(reference_output, hypothesis_list)

In [53]:
print(bleu)

31.166087542542304


In [59]:
df=pd.concat([pd.DataFrame(test_inputs),pd.DataFrame(reference_output),pd.DataFrame(hypothesis_list)],axis=1)
df.columns=['intent','Ground truth','Code output by Transformer']

In [60]:
df.head(10)

,intent,Ground truth,Code output by Transformer
0,send a signal `signal.SIGUSR1` to the current ...,"os.kill(os.getpid(), signal.SIGUSR1)",os .system ('some_command with args')
1,decode a hex string '4a4b4c' to UTF-8.,bytes.fromhex('4a4b4c').decode('utf-8'),int (var_1 (16 ))
2,check if all elements in list `myList` are ide...,all(x == myList[0] for x in myList),"(isinstance (var_1 ,var_2 )for var_1 in var_2 )"
3,format number of spaces between strings `Pytho...,"print('%*s : %*s' % (20, 'Python', 20, 'Very G...","int (var_1 .findall ('\\d+',var_2 ))"
4,How to convert a string from CP-1251 to UTF-8?,d.decode('cp1251').encode('utf8'),int (''.join (str (str (str (str (str ))))for ...
5,get rid of None values in dictionary `kwargs`,"res = {k: v for k, v in list(kwargs.items()) i...",[(x [0 ]for x in list (var_1 .items ()))if x ]
6,get rid of None values in dictionary `kwargs`,"res = dict((k, v) for k, v in kwargs.items() i...",[(x [0 ]for x in list (var_1 .items ()))if x ]
7,capture final output of a chain of system comm...,subprocess.check_output('ps -ef | grep somethi...,"subprocess .call (['shutdown','/s'])"
8,"concatenate a list of strings `['a', 'b', 'c']`",""""""""""""".join(['a', 'b', 'c'])",[int (var_1 )for var_1 in var_2 ]
9,find intersection data between series `s1` and...,pd.Series(list(set(s1).intersection(set(s2)))),var_1 .groupby (var_2 .columns ).size ()


In [ ]:
df.tail(20)

,0,0,0
480,"write line ""hi there"" to file `myfile`","f = open('myfile', 'w')\nf.write('hi there\n')...",os .write ('\n'.join (line ))
481,"write line ""Hello"" to file `somefile.txt`","with open('somefile.txt', 'a') as the_file:\n ...",os .write ('\n'.join (line ))
482,convert unicode string `s` to ascii,s.encode('iso-8859-15'),int (var_1 .encode ('hex'))
483,Django get maximum value associated with field...,AuthorizedEmail.objects.filter(group=group).or...,var_1 .objects .filter (var_1 =lambda x :x [0 ])
484,Find all numbers and dots from a string `text`...,"re.findall('Test([0-9.]*[0-9]+)', text)","re .findall ('#(\\w+)',re .findall ('\\d+',re ..."
485,python regex to find all numbers and dots fro...,"re.findall('Test([\\d.]*\\d+)', text)","re .findall ('[bcdfghjklmnpqrstvwxyz]+',re .fi..."
486,execute script 'script.ps1' using 'powershell....,"os.system('powershell.exe', 'script.ps1')","subprocess .call (['shutdown','/s'])"
487,Sort a list of tuples `b` by third item in the...,b.sort(key=lambda x: x[1][2]),"sorted (list (var_1 .items ()),key =lambda x :..."
488,get a list of all keys in Cassandra database `...,list(cf.get_range().get_keys()),[var_1 for var_1 in list (var_2 .items ())for ...
489,create a datetime with the current date & time,datetime.datetime.now(),datetime .datetime .datetime .now ().time ()


***Running the model for 8 heads instead of 16***

In [ ]:
INPUT_DIM = len(Input.vocab)
OUTPUT_DIM = len(Output.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

SRC_PAD_IDX = Input.vocab.stoi[Input.pad_token]
TRG_PAD_IDX = Output.vocab.stoi[Output.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

model.apply(initialize_weights);

LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_example = []
    val_example = []

    for i in range(train_df.shape[0]):
        try:
            ex = data.Example.fromlist([train_df.Text[i], train_df.Code[i]], fields)
            train_example.append(ex)
        except:
            pass

    for i in range(val_df.shape[0]):
        try:
            ex = data.Example.fromlist([val_df.Text[i], val_df.Code[i]], fields)
            val_example.append(ex)
        except:
            pass       

    train_data = data.Dataset(train_example, fields)
    valid_data =  data.Dataset(val_example, fields)

    BATCH_SIZE = 16
    train_iterator, valid_iterator = BucketIterator.splits((train_data, valid_data), batch_size = BATCH_SIZE, 
                                                                sort_key = lambda x: len(x.Input),
                                                                sort_within_batch=True, device = device)

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/model_8heads.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

here
iterator


  1%|          | 1/125 [00:00<01:28,  1.40it/s]

0


  2%|▏         | 2/125 [00:01<01:26,  1.43it/s]

1


...

123


100%|██████████| 125/125 [01:41<00:00,  1.23it/s]


124
printloss [23946.64365005493, 28099.210402965546, 31465.705114603043, 16899.59093928337, 19035.439305067062, 13895.85758972168, 9185.494194746017, 15657.79808974266, 13717.982485294342, 15147.957355499268, 26424.46057653427, 41431.839322566986, 23512.457335472107, 19549.225243330002, 29400.120200157166, 27244.857516288757, 26762.803444862366, 19799.682600975037, 30595.940391540527, 39132.33960413933, 26263.920011520386, 34365.95945549011, 17150.864629745483, 24547.04140472412, 19961.4720287323, 14277.099900245667, 32537.135642766953, 40011.457797288895, 28171.14630174637, 53382.29087471962, 29191.114277601242, 28399.969077587128, 22793.821383953094, 39636.9980134964, 14222.458661556244, 22629.633893013, 17421.059277057648, 10162.505598068237, 24160.81296157837, 23486.16611480713, 25701.629943847656, 19416.51820707321, 14681.343266963959, 27314.369387626648, 25678.318648338318, 14148.354683876038, 27832.855525493622, 22138.27694129944, 28810.14363837242, 30426.61593389511, 13463.179

100%|██████████| 25/25 [00:06<00:00,  4.14it/s]


Epoch: 05 | Time: 1m 47s
	Train Loss: 3.441 | Train PPL:  31.234
	 Val. Loss: 3.665 |  Val. PPL:  39.050


In [ ]:
SRC = Input
TRG = Output

In [ ]:
model.load_state_dict(torch.load('./model_8heads.pt'))

<All keys matched successfully>

In [ ]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['intent', 'rewritten_intent', 'snippet', 'question_id'],
        num_rows: 500
    })
})

In [ ]:
test_inputs,reference_output=creating_features(test_dataset)
hypothesis_list=[]
for i,input in enumerate(test_inputs):
  input=input.split(" ")
  translation, attention = translate_sentence(input, SRC, TRG, model, device)
  hypothesis_list.append(untokenize(translation[:-1]).decode('utf-8'))
  print(i)

bleu = evaluate_bleu(reference_output, hypothesis_list)
print(bleu)

['send', 'a', 'signal', '`signal.SIGUSR1`', 'to', 'the', 'current', 'process']
0
['decode', 'a', 'hex', 'string', "'4a4b4c'", 'to', 'UTF-8.']
1
['check', 'if', 'all', 'elements', 'in', 'list', '`myList`', 'are', 'identical']
2
['format', 'number', 'of', 'spaces', 'between', 'strings', '`Python`,', '`:`', 'and', '`Very', 'Good`', 'to', 'be', '`20`']
3
['How', 'to', 'convert', 'a', 'string', 'from', 'CP-1251', 'to', 'UTF-8?']
4
['get', 'rid', 'of', 'None', 'values', 'in', 'dictionary', '`kwargs`']
5
['get', 'rid', 'of', 'None', 'values', 'in', 'dictionary', '`kwargs`']
6
['capture', 'final', 'output', 'of', 'a', 'chain', 'of', 'system', 'commands', '`ps', '-ef', '|', 'grep', 'something', '|', 'wc', '-l`']
7
['concatenate', 'a', 'list', 'of', 'strings', "`['a',", "'b',", "'c']`"]
8
['find', 'intersection', 'data', 'between', 'series', '`s1`', 'and', 'series', '`s2`']
9
['sending', 'http', 'headers', 'to', '`client`']
10
['Format', 'a', 'datetime', 'string', '`when`', 'to', 'extract', 'dat

***Implementation with 6 layers and 16 heads***

In [ ]:
INPUT_DIM = len(Input.vocab)
OUTPUT_DIM = len(Output.vocab)
HID_DIM = 256
ENC_LAYERS = 6
DEC_LAYERS = 6
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_example = []
    val_example = []

    for i in range(train_df.shape[0]):
        try:
            ex = data.Example.fromlist([train_df.Text[i], train_df.Code[i]], fields)
            train_example.append(ex)
        except:
            pass

    for i in range(val_df.shape[0]):
        try:
            ex = data.Example.fromlist([val_df.Text[i], val_df.Code[i]], fields)
            val_example.append(ex)
        except:
            pass       

    train_data = data.Dataset(train_example, fields)
    valid_data =  data.Dataset(val_example, fields)

    BATCH_SIZE = 16
    train_iterator, valid_iterator = BucketIterator.splits((train_data, valid_data), batch_size = BATCH_SIZE, 
                                                                sort_key = lambda x: len(x.Input),
                                                                sort_within_batch=True, device = device)

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/model_16layers.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

here
iterator


  1%|          | 1/125 [00:00<01:42,  1.21it/s]

0


  2%|▏         | 2/125 [00:01<01:35,  1.29it/s]

1


...

123


100%|██████████| 125/125 [01:45<00:00,  1.19it/s]


124
printloss [34875.44212245941, 40901.449310302734, 46365.159487724304, 24263.46447944641, 26915.055956840515, 20187.06628704071, 13148.040266513824, 23315.73695898056, 21019.434699058533, 22138.69783782959, 39417.38041782379, 59059.6878657341, 34352.81895637512, 28249.447415828705, 42346.36480522156, 40031.79313659668, 39781.64600276947, 29262.917463302612, 45956.904870033264, 57681.26901912689, 38480.28057098389, 52340.90189409256, 24977.000675201416, 38559.37342596054, 29371.72689628601, 21426.742323875427, 47668.839344501495, 58450.43257713318, 39256.29832458496, 78062.33053159714, 41692.90338563919, 41145.817175388336, 33558.75666761398, 57734.71664714813, 20610.07026386261, 33765.04499864578, 25128.139088630676, 15432.102952003479, 35303.49424171448, 34564.11417579651, 36666.76954650879, 27803.096847057343, 22319.89004135132, 38745.41991233826, 37781.70889377594, 20905.446155548096, 40559.69719791412, 31174.780990600586, 43241.80304479599, 44412.63985347748, 19891.964626312256,

100%|██████████| 25/25 [00:06<00:00,  4.08it/s]


Epoch: 05 | Time: 1m 52s
	Train Loss: 5.027 | Train PPL: 152.506
	 Val. Loss: 5.036 |  Val. PPL: 153.826


In [ ]:
SRC = Input
TRG = Output

In [ ]:
test_inputs,reference_output=creating_features(test_dataset)
hypothesis_list=[]
for input in test_inputs:
  input=input.split(" ")
  translation, attention = translate_sentence(input, SRC, TRG, model, device)
  hypothesis_list.append(untokenize(translation[:-1]).decode('utf-8'))

bleu = evaluate_bleu(reference_output, hypothesis_list)

In [ ]:
print(bleu)

13.839435571817358


Reference - https://towardsdatascience.com/building-a-python-code-generator-4b476eec5804